In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-08-31 11:42:20--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-08-31 11:42:20--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc801d613c7dac8cb4969a99a66c.dl.dropboxusercontent.com/cd/0/inline/BVTZi7wBIbqipEgj0c712Imh3ISPh09dw83Qgk8FG8mpMEB1z_U7tK-6OSRJKw2iEsqnAJLiZR8MmQ4Q8qbVFAhl6CwdQXcREuRQ4a3Gc_mEgB9Ub-N4qCCwmgcaOl5N2sR5wFDwRy6mUYrW0vEPjTCH/file# [following]
--2021-08-31 11:42:20--  https://uc801d613c7dac8cb4969a99a66c.dl.dropboxusercontent.com/cd/0/inline/BVTZi7wBIbqipEgj0c712Imh3ISPh09dw83Qgk8FG8mpMEB1z_U

In [ ]:
# Let's unzip the file
!unzip -q "/content/movie-reviews-dataset.zip"

In [ ]:
from tensorflow.keras.preprocessing import  text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [ ]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, "<br />", " "), label),
  )

In [ ]:
train_data = prepareData("/content/movie-reviews-dataset/train")
test_data = prepareData("/content/movie-reviews-dataset/test")

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'... and yet, we were told, there was another hour and 20 minutes left to go.  Why, oh, why wasn\'t there an editor to tell the writer/director to snip, snip, snip? Apparently that writer/director has previously done shorts; as a short, this would have been okay. But the lack of dialogue starts to grate after twenty minutes. The lack of much music glares. The background noises (talking, traffic, and especially a ubiquitous helicopter) get old really fast. But the worst failure is in story. There is precious little beyond a short.  After an hour we saw variations of the same scene over and over again. I nearly screamed at the screen, "We get it, we get it!!!!!" It\'s amazing that after that left the theatre, we could drive home, watch the Daily Show and parts of the Colbert Report, get ready for bed,and know that the audience was STILL trapped in the theatre.  It\'s not enough to indulge your vision. Yo

In [ ]:
model =Sequential()

In [ ]:
model.add(Input(shape=(1,), dtype="string"))

In [ ]:
max_tokens= 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_len,
)

In [ ]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [ ]:
model.add(Embedding(max_tokens + 1, 128))

model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 70s 87ms/step - loss: 0.5347 - accuracy: 0.7258
Epoch 2/10
782/782 [==============================] - 67s 86ms/step - loss: 0.4380 - accuracy: 0.8003
Epoch 3/10
782/782 [==============================] - 67s 86ms/step - loss: 0.4080 - accuracy: 0.8149
Epoch 4/10
782/782 [==============================] - 66s 85ms/step - loss: 0.3858 - accuracy: 0.8271
Epoch 5/10
782/782 [==============================] - 67s 85ms/step - loss: 0.3639 - accuracy: 0.8410
Epoch 6/10
782/782 [==============================] - 68s 86ms/step - loss: 0.3492 - accuracy: 0.8478
Epoch 7/10
782/782 [==============================] - 68s 87ms/step - loss: 0.3387 - accuracy: 0.8531
Epoch 8/10
782/782 [==============================] - 69s 88ms/step - loss: 0.3196 - accuracy: 0.8649
Epoch 9/10
782/782 [==============================] - 69s 88ms/step - loss: 0.3089 - accuracy: 0.8677
Epoch 10/10
782/782 [==============================] - 69s 88ms/step - loss: 0.297

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 23s 28ms/step - loss: 0.5348 - accuracy: 0.7704


[0.5347712635993958, 0.7703999876976013]

In [ ]:
text = "This movis is bad"

In [ ]:
model.predict([text]) # 0 = Negative review, 1 = positive review

array([[0.02341992]], dtype=float32)

In [ ]:
text2 = "This movis is good"

In [ ]:
model.predict([text2]) # 0 = Negative review, 1 = positive review

array([[0.92961156]], dtype=float32)